## Prims algorithmus realized with heap and adjacent matrix

In [ ]:
# ---------- helpers ----------
def left(i): return 2*i
def right(i): return 2*i + 1
def parent(i): return i // 2

def _swap(A, i, j, pos):
    # print("i: ",i, " j: ",j, "\n")
    # print("A[i]: ",A[i], " A[j]: ",A[j], "\n")
    A[i], A[j] = A[j], A[i]
    # print("A[i]: ",A[i], " A[j]: ",A[j], "\n")

    _, vi = A[i]
    _, vj = A[j]
    # print("vi: ",vi, " vj: ",vj, "\n")
    pos[vi] = i
    pos[vj] = j

# ---------- core heap ops ----------
def min_heapify(A, i, n=None, pos=None):
    """A: [None,(key,v),...] ; 基于 A[i] 向下维护最小堆；可选 pos 以在交换时更新
    min-heap 的排序依据就是 key, 而这个 key 通常就是边的权重
    """
    if n is None: n = len(A) - 1
    while True:
        l, r = left(i), right(i)
        smallest = i
        if l <= n and A[l][0] < A[smallest][0]:
            smallest = l
        if r <= n and A[r][0] < A[smallest][0]:
            smallest = r
        if smallest != i:
            if pos is not None:
                _swap(A, i, smallest, pos)
            else:
                A[i], A[smallest] = A[smallest], A[i]
            i = smallest
        else:
            break

def build_min_heap(A, pos):
    """
    A 为 [None] + 列表((key,v),...)；会原地建堆并写入 pos[v] = index
    若 A 里尚未写 pos, 可先调用本函数建立。
    """
    n = len(A) - 1
    for i in range(1, n+1):
        _, v = A[i]
        pos[v] = i
    for i in range(n // 2, 0, -1):
        min_heapify(A, i, n, pos)

def build_min_heap_without_pos(A):
    """ 
    正确，能把 A 变成合法的最小堆
    但不能支持 decrease_key 里通过 pos[v] 找顶点位置
    """
    n = len(A) - 1
    for i in range(n // 2, 0, -1):
        min_heapify(A, i, n)


def heap_is_empty(A):
    return len(A) == 1

def heap_insert(A, pos, v, key):
    """插入 (key,v)：先放 +inf，再 decrease"""
    A.append((float("inf"), v))
    pos[v] = len(A) - 1
    heap_decrease_key(A, pos, v, key)

def heap_decrease_key(A, pos, v, new_key):
    """将顶点 v 的键减少为 new_key（要求 new_key <= 当前值）"""
    i = pos[v]
    if new_key > A[i][0]:
        raise ValueError("new_key is larger than current key")
    # 向上浮
    A[i] = (new_key, v)
    while i > 1 and A[parent(i)][0] > A[i][0]:
        _swap(A, i, parent(i), pos)
        i = parent(i)

def heap_extract_min(A, pos):
    """弹出最小项，返回 (key, v)"""
    if heap_is_empty(A):
        raise IndexError("extract_min from empty heap")
    min_item = A[1]
    last = A.pop()
    del pos[min_item[1]]
    if len(A) > 1:
        A[1] = last
        pos[last[1]] = 1
        min_heapify(A, 1, len(A)-1, pos)
    return min_item


In [10]:
def prim(adj, start):
    """
    adj: dict[u] -> list[(v, w)]  （无向图：需为每条边写两向）
    start: 起点
    返回：pred 前驱字典、MST 总权重 total
    """
    key   = {u: float("inf") for u in adj}
    pred  = {u: None for u in adj}
    in_T  = {u: False for u in adj}

    # 建堆：把所有顶点放进去，key=+inf；随后把 start 的 key 降到 0
    A   = [None] + [(key[u], u) for u in adj]
    pos = {}
    build_min_heap(A, pos)
    key[start] = 0.0
    heap_decrease_key(A, pos, start, 0.0)

    total = 0.0
    while not heap_is_empty(A):
        ku, u = heap_extract_min(A, pos)
        in_T[u] = True
        if pred[u] is not None:
            total += ku
        for v, w in adj[u]:
            if not in_T[v] and w < key[v]:
                key[v] = w
                pred[v] = u
                heap_decrease_key(A, pos, v, w)
    return pred, total

def print_mst(pred, adj, total):
    """
    pred: Prim 算法返回的前驱字典
    adj : 图的邻接表 (dict[u] -> list[(v, w)])
    total: MST 总权重
    """
    print("MST edges:")
    for v, u in pred.items():
        if u is not None:  # 跳过起点
            # 找到边权重
            w = next(weight for nbr, weight in adj[u] if nbr == v)
            print(f"  {u} -- {v} (weight {w})")
    
    
    print(f"\nTotal weight = {total}\n")
    mst_edges = [(u,v) for v,u in pred.items() if u is not None]
    print("MST edges:", mst_edges)


In [11]:
# 无向图（手动补双向）
adj = {
    'A': [('B', 4), ('H', 8)],
    'B': [('A', 4), ('C', 8), ('H', 11)],
    'C': [('B', 8), ('D', 7), ('F', 4), ('I', 2)],
    'D': [('C', 7), ('E', 9), ('F', 14)],
    'E': [('D', 9), ('F', 10)],
    'F': [('C', 4), ('D', 14), ('E', 10), ('G', 2)],
    'G': [('F', 2), ('H', 1), ('I', 6)],
    'H': [('A', 8), ('B', 11), ('G', 1), ('I', 7)],
    'I': [('C', 2), ('G', 6), ('H', 7)],
}
pred, total = prim(adj, 'A')
# pred 是生成树结构；total 是 MST 权重和

print_mst(pred, adj, total)


MST edges:
  A -- B (weight 4)
  F -- C (weight 4)
  C -- D (weight 7)
  D -- E (weight 9)
  G -- F (weight 2)
  H -- G (weight 1)
  A -- H (weight 8)
  C -- I (weight 2)

Total weight = 37.0

MST edges: [('A', 'B'), ('F', 'C'), ('C', 'D'), ('D', 'E'), ('G', 'F'), ('H', 'G'), ('A', 'H'), ('C', 'I')]


## 代码详解

In [13]:
#    A
#   / \
#  1   3
#  /     \
# B ——2—— C

A = {
    'A': [('B', 1), ('C', 3)],
    'B': [('A', 1), ('C', 2)],
    'C': [('A', 3), ('B', 2)],
}
print(A['A'])


[('B', 1), ('C', 3)]


###  _swap

In [18]:
# A = [None, (key, v), (key, v), ...]
# key = 当前顶点在堆里的优先级
# v = 顶点名字
# 堆是 1-indexed，所以 A[0] = None 占位

# pos[v] = i   # 顶点 v 在堆数组中的下标 i

# _, vi = A[i]
# A[i] 是一个 (key, v)
# _ 表示我们不关心 key，只要顶点名 v
# 所以 vi = A[i] 里的顶点，vj = A[j] 里的顶点

A = [ None, (3,'B'), (5,'C'), (7,'D') ]
pos = { 'B':1, 'C':2, 'D':3 }
_swap(A, 1, 3, pos)
print(pos)

A = [ None, (3,'B'), (5,'C'), (7,'D') ]
A[1], A[3] = A[3],A[1]
print(A)

i:  1  j:  3 

A[i]:  (3, 'B')  A[j]:  (7, 'D') 

A[i]:  (7, 'D')  A[j]:  (3, 'B') 

vi:  D  vj:  B 

{'B': 3, 'C': 2, 'D': 1}
[None, (7, 'D'), (5, 'C'), (3, 'B')]


### min_heapify

In [ ]:
# 输入：
# A 是 1-indexed 的堆数组：[None, (key,v), (key,v), ...]
# i 是要修复堆性质的节点位置
# n 是堆大小（如果不给，就取 len(A)-1）
# pos 是一个字典，维护顶点到堆下标的映射
# 目标：确保以 i 为根的子树满足最小堆性质（父节点的 key ≤ 子节点的 key）


# if pos is not None:
#     _swap(A, i, smallest, pos)
# else:
#     A[i], A[smallest] = A[smallest], A[i]
# i = smallest

# 1. 如果 pos is not None
# 说明你在用 Prim 算法的版本，堆里除了数组 A 之外，还维护了一个字典
# pos[v] = i   # 顶点 v 在堆数组中的下标 i
# 这样才能让 decrease_key(A, pos, v, new_key) 在 
# O(logn) 时间内找到顶点 v 的位置。
# 但是！一旦你交换了堆里两个元素的位置（A[i] 和 A[smallest]），
# pos 字典里的记录就过时了，必须更新。
# 所以调用：_swap(A, i, smallest, pos)
# 在交换的同时，同步更新 pos 字典
# 2. 如果 pos is None
# 说明你只是单纯想用堆，不需要 pos，比如排序或者检查堆结构。
# 这时候只需要交换数组里的两个元素，不必更新任何字典。
# 所以就写：
# A[i], A[smallest] = A[smallest], A[i]
# 3. 为什么要分支？
# 不分支的话，如果总是调用 _swap，那就要求一定传 pos；
# 但有时候你只想用最小堆排序，根本不需要 pos，这时传 pos=None，函数也应该能工作。
# 这个写法让 min_heapify 更通用：
# 有 pos → 维护 pos
# 没 pos → 只维护数组


### build_min_heap

In [27]:

A = [None, (3,'B'), (1,'A'), (2,'C'),(0,'D')]
pos = {}

print("Before build_min_heap:")
print("A =", A)
print("pos =", pos)

build_min_heap(A, pos)

print("\nAfter build_min_heap:")
print("A =", A)
print("pos =", pos)

A = [None, (3,'B'), (1,'A'), (2,'C'),(0,'D')]
print("Before build_min_heap_without_pos:")
print("A =", A)
print("pos =", pos)

build_min_heap_without_pos(A)

print("\nAfter build_min_heap_without_pos:")
print("A =", A)
print("pos =", pos)

Before build_min_heap:
A = [None, (3, 'B'), (1, 'A'), (2, 'C'), (0, 'D')]
pos = {}

After build_min_heap:
A = [None, (0, 'D'), (1, 'A'), (2, 'C'), (3, 'B')]
pos = {'B': 4, 'A': 2, 'C': 3, 'D': 1}
Before build_min_heap_without_pos:
A = [None, (3, 'B'), (1, 'A'), (2, 'C'), (0, 'D')]
pos = {'B': 4, 'A': 2, 'C': 3, 'D': 1}

After build_min_heap_without_pos:
A = [None, (0, 'D'), (1, 'A'), (2, 'C'), (3, 'B')]
pos = {'B': 4, 'A': 2, 'C': 3, 'D': 1}


### heap_extract_min

In [31]:
A = [None, (1,'A'), (4,'B'), (3,'C')]
print(A[1])
print(A[1][0])
print(A[1][1])
print(A.pop())
print(A)

(1, 'A')
1
A
(3, 'C')
[None, (1, 'A'), (4, 'B')]


In [ ]:
# heap_extract_min：把堆顶的最小元素删掉并返回，
# 然后用最后一个元素顶上来，
# 再调用 min_heapify 保证堆有序，同时维护好 pos 字典。

# min_item = A[1]
# 最小堆的堆顶 A[1] 就是最小值，记为 min_item。
# 注意：这是一个 (key, v) 元组。

# last = A.pop()
# 从数组末尾拿出最后一个元素。
# 堆里的元素总数减少 1。

# del pos[min_item[1]]
# 从位置字典 pos 删除掉刚刚取走的顶点。
# # 比如 min_item = (1,'A')，那就 del pos['A']

# 为什么 `heap_extract_min` 里要做 **`A[1] = last`**？

# 1. 堆顶空了，需要填补

# 在最小堆里，堆顶 A[1] 总是最小元素。
# 当我们取出 `A[1]`（最小值）后，这个位置就空了：


# Before extract:
# A = [None, (1,'A'), (3,'B'), (5,'C')]
# 取出 (1,'A')

# 这时候堆变成：


# A = [None, ?, (3,'B'), (5,'C')]


# 堆顶不能留空，否则数组就断裂了。


# 2. 为什么用最后一个元素来填补？

# 堆的存储方式是**完全二叉树**（complete binary tree），也就是：

# * 每一层从左到右填满，只有最后一层可能不满。
# * 这样才能保证树的高度 = O(log n)。

# 如果直接把堆顶删掉，不用最后一个元素来补，就会在数组中间留下“空洞”，破坏完全二叉树的结构。

# 所以标准做法是：

# * 把数组的最后一个元素拿来填补堆顶。
# * 这样数组还是连续的，没有空位。

# 3. 接下来要做什么？

# 当然，最后一个元素随便放到堆顶后，堆性质可能会破坏：

# 原来： (1,'A') 是最小
# 放上来： (5,'C') → 可能比孩子大


# 所以要调用 `min_heapify(A,1)`，把它往下沉，恢复最小堆。

# 4. 动态例子

# 初始：

# A = [None, (1,'A'), (3,'B'), (5,'C')]

# * 最小是 `(1,'A')`，我们要删掉它。
# * 最后一个元素是 `(5,'C')`。
# * 把它移到堆顶：

# 临时状态: [None, (5,'C'), (3,'B')]

# * 然后调用 `min_heapify(A,1)`：

#   * 比较 (5,'C') 和孩子 (3,'B') → 交换
#   * 结果：

# A = [None, (3,'B'), (5,'C')]

# `A[1] = last` 的目的：

# * 保持堆的数组结构连续（不留空洞），
# * 维持完全二叉树的性质。
#   之后用 `min_heapify` 来恢复堆序。


### prim

In [32]:
a = float("inf")

In [33]:
a

inf

In [34]:
b = float("-inf")

In [35]:
b

-inf

In [36]:
A = {
    'A': [('B', 1), ('C', 3)],
    'B': [('A', 1), ('C', 2)],
    'C': [('A', 3), ('B', 2)],
}
key   = {u: float("inf") for u in A}
key

{'A': inf, 'B': inf, 'C': inf}

In [42]:
key   = {u: float("inf") for u in A}
print("key: ", key)
A = {
    'A': [('B', 1), ('C', 3)],
    'B': [('A', 1), ('C', 2)],
    'C': [('A', 3), ('B', 2)],
}
B = [None] + [(key[u], u) for u in A]

for u in A:
    print(u)
B

key:  {'A': inf, 'B': inf, 'C': inf}
A
B
C


[None, (inf, 'A'), (inf, 'B'), (inf, 'C')]

In [44]:
A = {
    'A': [('B', 4), ('H', 8)],
    'B': [('A', 4), ('C', 8), ('H', 11)],
    'C': [('B', 8), ('D', 7), ('F', 4), ('I', 2)],
    'D': [('C', 7), ('E', 9), ('F', 14)],
    'E': [('D', 9), ('F', 10)],
    'F': [('C', 4), ('D', 14), ('E', 10), ('G', 2)],
    'G': [('F', 2), ('H', 1), ('I', 6)],
    'H': [('A', 8), ('B', 11), ('G', 1), ('I', 7)],
    'I': [('C', 2), ('G', 6), ('H', 7)],
}
key   = {u: float("inf") for u in A}
print("key: ", key)
B = [None] + [(key[u], u) for u in A]

B

key:  {'A': inf, 'B': inf, 'C': inf, 'D': inf, 'E': inf, 'F': inf, 'G': inf, 'H': inf, 'I': inf}


[None,
 (inf, 'A'),
 (inf, 'B'),
 (inf, 'C'),
 (inf, 'D'),
 (inf, 'E'),
 (inf, 'F'),
 (inf, 'G'),
 (inf, 'H'),
 (inf, 'I')]